In [10]:
# coding=utf-8
"""
This module contains the tests which ensure that data validation checks always occur when running a bancor3-related arb_mode.
"""
from fastlane_bot import Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
import subprocess, os, sys
import pytest
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.14 (23/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Setup

In [11]:
def find_main_py():
    # Start at the directory of the current script
    cwd = os.path.abspath(os.path.join(os.getcwd()))
    
    print(f"Searching for main.py in {cwd}")
    while True:
        # Check if main.py exists in the current directory
        if "main.py" in os.listdir(cwd):
            return cwd  # Found the directory containing main.py
        else:
            # If not, go up one directory
            new_cwd = os.path.dirname(cwd)

            # If we're already at the root directory, stop searching
            if new_cwd == cwd:
                raise FileNotFoundError("Could not find main.py in any parent directory")

            cwd = new_cwd
       
       
def run_command(arb_mode, expected_log_line):
    
    # Find the correct path to main.py
    main_script_path = find_main_py()
    print(f"Found main.py in {main_script_path}")
    main_script_path = main_script_path + "/main.py"

    # Run the command
    cmd = [
        "python",
        main_script_path,
        f"--arb_mode={arb_mode}",
        "--default_min_profit_bnt=60",
        "--limit_bancor3_flashloan_tokens=False",
        "--use_cached_events=True",
        "--logging_path=fastlane_bot/data/",
        "--timeout=80",
        "--blockchain=ethereum"
    ]
    subprocess.Popen(cmd)
        
    # Wait for the expected log line to appear
    found = False
    result = subprocess.run(cmd, text=True, capture_output=True, check=True, timeout=120)

    # Check if the expected log line is in the output
    if expected_log_line in result.stderr or expected_log_line in result.stdout:
        found = True

    if not found:
        pytest.fail("Expected log line was not found within 1 minute")  # If we reach this point, the test has failed

## Test Data Validation For b3_two_hop

In [12]:
expected_log_line = "Transactions will be required to pass data validation for"
arb_mode = "b3_two_hop"
run_command(arb_mode=arb_mode, expected_log_line=expected_log_line)

Searching for main.py in /Users/mikewcasale/Documents/GitHub/fastlane-bot/resources/NBTest
Found main.py in /Users/mikewcasale/Documents/GitHub/fastlane-bot


2023-07-31 06:16:43,933 [fastlane:INFO] - Using mainnet config
2023-07-31 06:16:43,934 [fastlane:INFO] - Running data fetching for exchanges: ['carbon_v1', 'bancor_v3', 'uniswap_v3', 'uniswap_v2', 'sushiswap_v2']
2023-07-31 06:16:44,195 [fastlane:INFO] - Time taken to add initial pools: 0.0898439884185791
2023-07-31 06:16:44,405 [fastlane:INFO] - Fetching events from 17811229 to 17813229... 0
2023-07-31 06:16:44,405 [fastlane:INFO] - Using cached events
2023-07-31 06:16:44,406 [fastlane:INFO] - Found 25 new events
2023-07-31 06:16:46,864 [fastlane:INFO] - Updating carbon pools w/ multicall...
2023-07-31 06:16:47,705 [fastlane:INFO] - Fetched 251 carbon strategies in 0.8403658866882324 seconds
2023-07-31 06:16:52,241 [fastlane:INFO] - Updated 251 carbon strategies info in 4.536462068557739 seconds
2023-07-31 06:17:09,266 [fastlane:INFO] - Initializing the bot...
2023-07-31 06:17:09,382 [fastlane:INFO] - State has changed...
2023-07-31 06:17:09,387 [fastlane:INFO] - Removed 122 unmapped 

timeout: 45
timeout: 45


2023-07-31 06:17:37,063 [fastlane:INFO] - Timeout hit... stopping bot
